# Titel
namen

teamnummer

## Introductie
Persvrijheid is een principe dat stelt dat de pers vrij is om te schrijven, zeggen en publiceren wat zij willen. De overheid is in beginsel verboden om censuur toe te passen en mag niet vooraf controleren wat wel of niet gezegd mag worden. In Nederland is persvrijheid, samen met het recht op meningsuiting, vastgelegd in artikel 7 van de Grondwet. Ook in andere landen en binnen Europees verband zijn er wetten die persvrijheid en vrijheid van meningsuiting beschermen.

Om te verzekeren dat journalisten hun werk veilig kunnen doen, hebben zij een beschermde status van het Europese Hof voor de Rechten van de Mens (EHRM). Journalisten brengen informatie naar buiten over algemeen belangrijke zaken en zijn daarom van cruciaal belang voor de democratie. Hoewel er vrijheid is om te publiceren wat men wil, betekent dit niet dat dit onbestraft kan gebeuren. Wanneer publicaties discriminerend zijn of aanzetten tot opruiing, kan iemand aangeklaagd en veroordeeld worden. Dit gebeurt echter pas achteraf; vóór publicatie is censuur niet toegestaan.

In veel landen is persvrijheid echter niet gegarandeerd. In landen met een autocratisch regime is de vrijheid van meningsuiting vaak beperkt of volledig afwezig. Ook in landen waar conflicten gaande zijn, zijn er mogelijk meerdere machten aan de leiding, of lijkt er niemand aan de macht te zijn. In zulke situaties is het moeilijk om te waarborgen dat de wet wordt nageleefd.

Deze data-analyse zal onderzoeken of de mate van persvrijheid wordt beïnvloed door conflicten of de mate van democratie. Door gebruik te maken van data van Reporters Without Borders (RSF), The Armed Conflict Location & Event Data Project (ACLED) en Our World in Data, zullen we scores en ranglijsten analyseren rondom persvrijheid, democratie en conflicten wereldwijd. Het perspectief dat conflicten de persvrijheid verminderen en het perspectief dat autocratische bestuursvormen de persvrijheid verminderen, worden hier onderzocht.

## Dataset and Preprocessing

tekst

## First perspective

tekst

In [11]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as py
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np

### Eerste argument van eeste perspectief

Om meerdere landen te selecteren ctrl indrukken

In [10]:
data_pers_ranking_19 = pd.read_csv('./data/2019.csv', sep=';')
data_pers_ranking_23 = pd.read_csv('./data/2023.csv', sep=';')

data_conflict = pd.read_excel('./data/01-2024-conflic-index.xlsx')

data_pers_ranking_19.rename(columns={'EN_country': 'country', 'Rank N': 'Rank_2019'}, inplace=True)
data_pers_ranking_23.rename(columns={'Country_EN': 'country', 'Rank': 'Rank_2023'}, inplace=True)
data_conflict.rename(columns={'Country': 'country', 'Index Ranking 2023': 'Conflict_Rank_2023', 'Change in ranking since 2019': 'Change_Conflict'}, inplace=True)

data_pers_combined = pd.merge(data_pers_ranking_19[['country', 'Rank_2019']], 
                              data_pers_ranking_23[['country', 'Rank_2023']], 
                              on='country', 
                              how='inner')


data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']


def filter_countries(text, dataset):
    if dataset == 'Conflict':
        return [country for country in data_conflict['country'] if text.lower() in country.lower()]
    else:
        return [country for country in data_pers_combined['country'] if text.lower() in country.lower()]


def plot_countries_ranking(countries):
    if not countries:
        print("Selecteer minstens een land")
        return
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Conflict Index', 'Press Freedom Index'))
    
    for country in countries:

        selected_data_conflict = data_conflict[data_conflict['country'] == country]
        
        if len(selected_data_conflict) > 0:
            ranking_2019 = selected_data_conflict['Conflict_Rank_2019'].values[0]
            ranking_2023 = selected_data_conflict['Conflict_Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            if change == 0:
                color = 'blue'
            elif change > 0:
                color = 'green'
            else:
                color = 'red' 
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Conflict ({ranking_2019} -> {ranking_2023})'
            ), row=1, col=1)
        
        selected_data_pers = data_pers_combined[data_pers_combined['country'] == country]
        
        if len(selected_data_pers) > 0:
            ranking_2019 = selected_data_pers['Rank_2019'].values[0]
            ranking_2023 = selected_data_pers['Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            if change == 0:
                color = 'blue'
            elif change > 0:
                color = 'red'
            else:
                color = 'green' 
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Press Freedom ({ranking_2019} -> {ranking_2023})'
            ), row=2, col=1)
        
    if len(countries) == 1:
        title = f'Verandering in Ranking voor {countries[0]}'
    elif len(countries) == 2:
        title = f'Verandering in Ranking voor {" en ".join(countries)}'
    elif len(countries) > 5:
        title = f'Verandering in Ranking voor {len(countries)} landen'
    else:
        title = f'Verandering in Ranking voor {", ".join(countries)}'
        
    fig.update_layout(
        height=800,
        title_text=title,
        showlegend=True,
    )
    
    fig.update_xaxes(title_text='Jaar', row=1, col=1)
    fig.update_xaxes(title_text='Jaar', row=2, col=1)
    fig.update_yaxes(title_text='Conflict Index Ranking', row=1, col=1)
    fig.update_yaxes(title_text='Press Freedom Index Ranking', row=2, col=1)
    
    fig.show()

dataset_dropdown = widgets.Dropdown(
    options=['Conflict', 'Press Freedom'],
    value='Conflict',
    description='Dataset:'
)

country_dropdown = widgets.SelectMultiple(
    options = data_conflict['country'].unique().tolist() if dataset_dropdown.value == 'Conflict' else data_pers_combined['country'].unique().tolist(),
    description = 'Landen:',
    disabled = False,
    layout = widgets.Layout(height='200px', width='400px'),
)

search_text = widgets.Text(
    value = '',
    placeholder = 'Zoek landen',
    description = 'Zoeken:',
    disabled = False
)

def handle_search(change):
    search_results = filter_countries(change.new, dataset_dropdown.value)
    
    selected_countries = list(country_dropdown.value)
    country_dropdown.options = search_results
    
    country_dropdown.value = [country for country in selected_countries if country in search_results]

def update_country_dropdown(change):
    search_results = filter_countries(search_text.value, change.new)
    country_dropdown.options = search_results
    country_dropdown.value = []

search_text.observe(handle_search, names='value')
dataset_dropdown.observe(update_country_dropdown, names='value')

display(dataset_dropdown)
display(search_text)

interact(plot_countries_ranking, countries=country_dropdown)

Dropdown(description='Dataset:', options=('Conflict', 'Press Freedom'), value='Conflict')

Text(value='', description='Zoeken:', placeholder='Zoek landen')

interactive(children=(SelectMultiple(description='Landen:', layout=Layout(height='200px', width='400px'), opti…

<function __main__.plot_countries_ranking(countries)>

tekst

In [12]:
df = pd.DataFrame(data_conflict)

data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']

conditions = [
    (df['Change_Conflict'] > 0),
    (df['Change_Conflict'] < 0),
    (df['Change_Conflict'] == 0)
]
choices = ['Verslechterd', 'Verbeterd', 'Gelijk gebleven']
df['Status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = df['Status'].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Conflict Index Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

tekst

In [13]:
data_pers_combined['Change'] = data_pers_combined['Rank_2023'] - data_pers_combined['Rank_2019']

conditions = [
    (data_pers_combined['Change'] > 0),
    (data_pers_combined['Change'] < 0),
    (data_pers_combined['Change'] == 0)
]
choices = ['Verbeterd', 'Verslechterd', 'Gelijk gebleven']
data_pers_combined['Status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = data_pers_combined['Status'].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Persvrijheidsindex Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

### Tweede argument: eerste perspectief

tekst

In [18]:
conflict_data = pd.read_excel('./data/01-2024-conflic-index.xlsx')
data_conflict_ranking = conflict_data[['country', 'Index Ranking 2023']]

data_pers = pd.read_csv('./data/2023.csv', sep=';')
data_pers.rename(columns = {'Country_EN':'country'}, inplace=True)

data_pers_ranking = data_pers[['country', 'Rank']]

data_both = pd.merge(data_pers_ranking, data_conflict_ranking, on='country', how='inner')

def determine_color(rank):
    if rank <= 10:
        return 'red'
    elif rank <= 30:
        return 'orange'
    elif rank <= 50:
        return 'yellow'
    else:
        return 'blue'

df = pd.DataFrame(data_both)
df['Color'] = df['Index Ranking 2023'].apply(determine_color)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Index Ranking 2023'],
    y=df['Rank'],
    mode='markers',
    text=df['country'],
    marker=dict(
        color=df['Color'],
        size=10
    ),
    name = 'Land',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='red', size=10),
    legendgroup='group1',
    showlegend=True,
    name='Extreem (Top 10)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='orange', size=10),
    legendgroup='group2',
    showlegend=True,
    name='Hoog (11-30)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='yellow', size=10),
    legendgroup='group3',
    showlegend=True,
    name='Turbulent (31-50)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='blue', size=10),
    legendgroup='group4',
    showlegend=True,
    name='Laag/inactief (>50)'
))

fig.add_trace(go.Scatter(
    x=[min(df['Index Ranking 2023']), max(df['Index Ranking 2023'])],
    y=[max(df['Rank']), min(df['Rank'])],
    mode='lines',
    line=dict(color='black'),
    name='Trend lijn'
))

fig.update_layout(
    title = 'Press Freedom Index Ranking vs Conflict Index Ranking (2023)',
    xaxis_title = 'Conflict Index Ranking (Index Ranking 2023)',
    yaxis_title = 'Press Freedom Index Ranking (Rank)',
    showlegend = True,
    height = 700
)

fig.show()

correlation = df['Index Ranking 2023'].corr(df['Rank'])

print(f"Correlatie tussen de conflict index ranking en de persvrijheid index ranking: {correlation}")

Correlatie tussen de conflict index ranking en de persvrijheid index ranking: -0.42573893484944786


tekst

In [19]:
df['Color'] = df['Rank'].apply(determine_color)

color_counts = df['Color'].value_counts()

color_labels = {
    'red': 'Extreem',
    'orange': 'Hoog',
    'yellow': 'Turbulent',
    'blue': 'laag/inactief'
}


labels = [color_labels[color] for color in color_counts.index]


fig = go.Figure(data=[go.Pie(
    labels = labels,
    values = color_counts.values,
    marker = dict(colors = color_counts.index)
)])

fig.update_layout(
    title = 'Aantal landen met bepaalde conflict status (index level) ',
    legend_title = 'Index level'
)

fig.show()

Tekst

### Hetzelfde voor het tweede perspectief

Tekst

## Reflectie

Tekst

## Work distribution

Tekst

## Referenties

Tekst